## Process the data

In [1]:
# !/bin/bash

# process text  
#echo 'tokenising text'
# first argument is the language to use; second arg is the path to the python script; third arg is the path to where get the data from; 
!python3 tokenize_texts4mallet.py titles-abstracts-dir.txt clean-data/

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/zhaozitong/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/zhaozitong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/zhaozitong/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
['titles-abstracts.csv UKRI']
titles-abstracts.csv

chunk 0

chunk 1

chunk 2

chunk 3

chunk 4

chunk 5

chunk 6

chunk 7

chunk 8

chunk 9

chunk 10



In [ ]:
# !/bin/bash

!echo filtering data
# call helper script to filter minimum number of tokens
# first argument below is the language of the script being called; second argument is the path to where the script is located; thrid argument is where to get the
# input data from; fourth argument is where to save the filtered data to. 
!python3 filter-tokens.py clean-data/titles-abstracts-tokenized.csv clean-data/


## Add the label

In [1]:
import pandas as pd

df = pd.read_csv('clean-data/titles-abstracts-tokenized-filtered.csv', header=None, names=['Column'])
df['ProjectId'] = df['Column'].apply(lambda x: x[:x.find('UKRI')].strip())
df['words'] = df['Column'].apply(lambda x: x[x.find('UKRI')+4:].strip())

df.drop('Column', axis=1, inplace=True)    
df.to_csv('temp_titles-abstracts-tokenized-filtered.csv', index=False)





In [29]:
from datetime import datetime
import pandas as pd
import numpy as np

df = pd.read_csv('clean-data/UKRI-project-metadata.csv',usecols=['ProjectId','StartDate','EndDate','FundingAmount','FundingBody'])
df['ProjectId']= df['ProjectId'].str.replace(" ", "-", regex=True)
df['FundingBody']= df['FundingBody'].str.replace(" ", "-", regex=True)
df = df[df['FundingAmount'] > 0].reset_index(drop=True)

df['ProjectId']= df1['ProjectId'].str.replace(" ", "-", regex=True)
df['Duration'] = ''
df['Fundingpy'] = ''

df['StartDate'] = pd.to_datetime(df['StartDate'], format="%d/%m/%Y", errors='coerce')
df['EndDate'] = pd.to_datetime(df['EndDate'], format="%d/%m/%Y", errors='coerce')


df['Duration'] = df['EndDate'] - df['StartDate']
df = df.drop(['StartDate', 'EndDate'], axis=1)
df.dropna(inplace=True)

df['FundingAmount'] = df['FundingAmount'].astype(int)
df['Duration'] = df['Duration'].astype(str).str.replace(' days', '')
df['Duration'] = df['Duration'].astype(int)
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')
df['Fundingpy'] = df['FundingAmount'] / (df['Duration']/365)

df['Fundingpy'].replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(subset=['Fundingpy'], inplace=True)
df['Fundingpy']=df['Fundingpy'].astype(int)
df = df[df['Duration'] > 0].reset_index(drop=True)


df.to_csv('duration-funding.csv', index = False)

df1 =pd.read_csv("temp_titles-abstracts-tokenized-filtered.csv")

merged_df = pd.merge(df1, df, on='ProjectId', how='left')

merged_df = merged_df[['ProjectId', 'Duration', 'FundingBody','FundingAmount','Fundingpy','words']]

merged_df.dropna(inplace=True)


merged_df[['ProjectId','FundingBody','words']].to_csv('funder-titles-abstracts-tokenized-filtered.csv',index=False)
merged_df[['ProjectId','Duration','words']].to_csv('duration-titles-abstracts-tokenized-filtered.csv',index=False)
merged_df[['ProjectId','Fundingpy','words']].to_csv('funding-titles-abstracts-tokenized-filtered.csv',index=False)

merged_df[['ProjectId','Fundingpy','FundingBody','Duration','words','FundingAmount']].to_csv('label-titles-abstracts-tokenized-filtered.csv',index=False)







